## Query structures from code
### bla

In [2]:
import numpy as np
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import load_node, Node, Code, Calculation, CalculationFactory
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.querybuilder import QueryBuilder



In [3]:
# Which codes do I have, and how many calculations have I launched with each one of them?
for code, in QueryBuilder().append(Code).all():
    print code.id, code.label, QueryBuilder().append(Code, filters={'id':code.id}).append(Calculation).count()

1088 pw-SVN-piz-daint 444
4681 pw-5.1 21


In [4]:
# Now I want to find out all vc-relax QE-calculation that were run using the Code with label pw-SVN-piz-daint
# on a single machine.
# I am interested in the average walltime used by these calculations

qb = QueryBuilder()
qb.append(Code, filters={'label':'pw-SVN-piz-daint'})
qb.append(CalculationFactory('quantumespresso.pw'),tag='calculation', 
          filters={'attributes.jobresource_params.num_machines':1})
qb.append(ParameterData, input_of='calculation',
          filters={'attributes.SYSTEM.ecutwfc':{'>=':50}, 'attributes.CONTROL.calculation':'vc-relax'})
qb.append(ParameterData, output_of='calculation', project=('attributes.wall_time_seconds'))

print "Average walltime (in seconds):", np.mean(zip(*qb.all())[0])


Average walltime (in seconds): 550.15045045
